## Setup

In [1]:
import json
import re
import datetime
from collections import OrderedDict

import pandas as pd
import pandas.io.json as pj
from IPython.core.debugger import set_trace

import tbap.api as api
import tbap.server as server

In [2]:
key = "6Sx6a2gOyNWwNNTAd9UYBOpWp7MxaNrZgO4CqConwJwwDooKQV3DbZXvnk8TBL7A"
username = "stacy.k.irwin@gmail.com"
sn = api.Session(username, key)
snj = api.Session(username, key, data_format="json")

# Display floating point numbers in fixed format within dataframes.
pd.options.display.float_format = '{:f}'.format

### get_matches()

In [3]:
sn.time_zone = "America/Chicago"
matches = api.get_matches(sn, "2017tur")

In [4]:
matches.head(12)

alliance  surrogate  score  disqualified  \
key          team_key                                            
2017tur_f1m1 frc2046      blue      False    503         False   
             frc1318      blue      False    503         False   
             frc1595      blue      False    503         False   
             frc1296       red      False    271         False   
             frc2473       red      False    271         False   
             frc1323       red      False    271         False   
2017tur_f1m2 frc2046      blue      False    461         False   
             frc1318      blue      False    461         False   
             frc1595      blue      False    461         False   
             frc1296       red      False    424         False   
             frc2473       red      False    424         False   
             frc1323       red      False    424         False   

                                    actual_time comp_level event_key  \
key          team_key                                                  
2017tur_f1m1 frc2046  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1318  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1595  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1296  2017-04-22 12:18:21-05:00          f   2017tur   
             frc2473  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1323  2017-04-22 12:18:21-05:00          f   2017tur   
2017tur_f1m2 frc2046  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1318  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1595  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1296  2017-04-22 12:35:04-05:00          f   2017tur   
             frc2473  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1323  2017-04-22 12:35:04-05:00          f   2017tur   

                       match_number          post_result_time  \
key          team_key                                           
2017tur_f1m1 frc2046              1 2017-04-22 12:22:52-05:00   
             frc1318              1 2017-04-22 12:22:52-05:00   
             frc1595              1 2017-04-22 12:22:52-05:00   
             frc1296              1 2017-04-22 12:22:52-05:00   
             frc2473              1 2017-04-22 12:22:52-05:00   
             frc1323              1 2017-04-22 12:22:52-05:00   
2017tur_f1m2 frc2046              2 2017-04-22 12:37:52-05:00   
             frc1318              2 2017-04-22 12:37:52-05:00   
             frc1595              2 2017-04-22 12:37:52-05:00   
             frc1296              2 2017-04-22 12:37:52-05:00   
             frc2473              2 2017-04-22 12:37:52-05:00   
             frc1323              2 2017-04-22 12:37:52-05:00   

                                 predicted_time       ...         \
key          team_key                                 ...          
2017tur_f1m1 frc2046  2017-04-22 12:19:01-05:00       ...          
             frc1318  2017-04-22 12:19:01-05:00       ...          
             frc1595  2017-04-22 12:19:01-05:00       ...          
             frc1296  2017-04-22 12:19:01-05:00       ...          
             frc2473  2017-04-22 12:19:01-05:00       ...          
             frc1323  2017-04-22 12:19:01-05:00       ...          
2017tur_f1m2 frc2046  2017-04-22 12:34:01-05:00       ...          
             frc1318  2017-04-22 12:34:01-05:00       ...          
             frc1595  2017-04-22 12:34:01-05:00       ...          
             frc1296  2017-04-22 12:34:01-05:00       ...          
             frc2473  2017-04-22 12:34:01-05:00       ...          
             frc1323  2017-04-22 12:34:01-05:00       ...          

                       teleopFuelHigh teleopFuelLow teleopFuelPoints  \
key          team_key                                                  
2017tur_f1m1 frc2046               63             0               21   
        

In [5]:
matches.loc["2017tur_f1m2"].at["frc1318", "score"]

461

In [6]:
matches.xs("frc1318", level=1).head()

,alliance,surrogate,score,disqualified,actual_time,comp_level,event_key,match_number,post_result_time,predicted_time,...,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,totalPoints,touchpadFar,touchpadMiddle,touchpadNear
key,,,,,,,,,,,,,,,,,,,,,
2017tur_f1m1,blue,False,503,False,2017-04-22 12:18:21-05:00,f,2017tur,1,2017-04-22 12:22:52-05:00,2017-04-22 12:19:01-05:00,...,63,0,21,391,120,150,503,ReadyForTakeoff,ReadyForTakeoff,ReadyForTakeoff
2017tur_f1m2,blue,False,461,False,2017-04-22 12:35:04-05:00,f,2017tur,2,2017-04-22 12:37:52-05:00,2017-04-22 12:34:01-05:00,...,39,0,13,383,120,150,461,ReadyForTakeoff,ReadyForTakeoff,ReadyForTakeoff
2017tur_qf4m1,red,False,491,False,2017-04-22 10:00:00-05:00,qf,2017tur,1,2017-04-22 10:02:42-05:00,2017-04-22 09:59:33-05:00,...,40,0,13,383,120,150,491,ReadyForTakeoff,ReadyForTakeoff,ReadyForTakeoff
2017tur_qf4m2,red,False,484,False,2017-04-22 10:47:36-05:00,qf,2017tur,2,2017-04-22 10:50:16-05:00,2017-04-22 10:47:25-05:00,...,30,1,10,340,80,150,484,ReadyForTakeoff,ReadyForTakeoff,ReadyForTakeoff
2017tur_qm100,red,False,307,False,2017-04-21 16:43:05-05:00,qm,2017tur,100,2017-04-21 16:45:49-05:00,2017-04-21 16:42:01-05:00,...,5,0,1,231,80,150,307,ReadyForTakeoff,ReadyForTakeoff,ReadyForTakeoff


In [7]:
matches.shape

(770, 47)

In [8]:
matches = api.get_matches(sn, "2017tur", response="simple")

In [12]:
matches.head(12)

alliance  surrogate  score  disqualified  \
key          team_key                                            
2017tur_f1m1 frc2046      blue      False    503         False   
             frc1318      blue      False    503         False   
             frc1595      blue      False    503         False   
             frc1296       red      False    271         False   
             frc2473       red      False    271         False   
             frc1323       red      False    271         False   
2017tur_f1m2 frc2046      blue      False    461         False   
             frc1318      blue      False    461         False   
             frc1595      blue      False    461         False   
             frc1296       red      False    424         False   
             frc2473       red      False    424         False   
             frc1323       red      False    424         False   

                                    actual_time comp_level event_key  \
key          team_key                                                  
2017tur_f1m1 frc2046  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1318  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1595  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1296  2017-04-22 12:18:21-05:00          f   2017tur   
             frc2473  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1323  2017-04-22 12:18:21-05:00          f   2017tur   
2017tur_f1m2 frc2046  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1318  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1595  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1296  2017-04-22 12:35:04-05:00          f   2017tur   
             frc2473  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1323  2017-04-22 12:35:04-05:00          f   2017tur   

                       match_number            predicted_time  set_number  \
key          team_key                                                       
2017tur_f1m1 frc2046              1 2017-04-22 12:19:01-05:00           1   
             frc1318              1 2017-04-22 12:19:01-05:00           1   
             frc1595              1 2017-04-22 12:19:01-05:00           1   
             frc1296              1 2017-04-22 12:19:01-05:00           1   
             frc2473              1 2017-04-22 12:19:01-05:00           1   
             frc1323              1 2017-04-22 12:19:01-05:00           1   
2017tur_f1m2 frc2046              2 2017-04-22 12:34:01-05:00           1   
             frc1318              2 2017-04-22 12:34:01-05:00           1   
             frc1595              2 2017-04-22 12:34:01-05:00           1   
             frc1296              2 2017-04-22 12:34:01-05:00           1   
             frc2473              2 2017-04-22 12:34:01-05:00           1   
             frc1323              2 2017-04-22 12:34:01-05:00           1   

                                           time winning_alliance  
key          team_key                                             
2017tur_f1m1 frc2046  2017-04-22 12:00:00-05:00             blue  
             frc1318  2017-04-22 12:00:00-05:00             blue  
             frc1595  2017-04-22 12:00:00-05:00             blue  
             frc1296  2017-04-22 12:00:00-05:00             blue  
             frc2473  2017-04-22 12:00:00-05:00             blue  
             frc1323  2017-04-22 12:00:00-05:00             blue  
2017tur_f1m2 frc2046  2017-04-22 12:13:00-05:00             blue  
             frc1318  2017-04-22 12:13:00-05:00             blue  
             frc1595  2017-04-22 12:13:00-05:00             blue  
             frc1296  2017-04-22 12:13:00-05:00             blue  
             frc2473  2017-04-22 12:13:00-05:00             blue  
             frc1323  2017-04-22 12:13:00-05:00             blue

In [14]:
matches.shape

(770, 12)

In [9]:
matches = api.get_matches(sn, "2017tur", response="keys")

In [10]:
matches.head()

,key
0,2017tur_f1m1
1,2017tur_f1m2
2,2017tur_qf1m1
3,2017tur_qf1m2
4,2017tur_qf2m1


In [18]:
matches = api.get_matches(sn, "2017tur", "frc1318")

In [19]:
matches.head()

alliance  surrogate  score  disqualified  \
key          team_key                                            
2017tur_f1m1 frc2046      blue      False    503         False   
             frc1318      blue      False    503         False   
             frc1595      blue      False    503         False   
             frc1296       red      False    271         False   
             frc2473       red      False    271         False   

                                    actual_time comp_level event_key  \
key          team_key                                                  
2017tur_f1m1 frc2046  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1318  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1595  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1296  2017-04-22 12:18:21-05:00          f   2017tur   
             frc2473  2017-04-22 12:18:21-05:00          f   2017tur   

                       match_number          post_result_time  \
key          team_key                                           
2017tur_f1m1 frc2046              1 2017-04-22 12:22:52-05:00   
             frc1318              1 2017-04-22 12:22:52-05:00   
             frc1595              1 2017-04-22 12:22:52-05:00   
             frc1296              1 2017-04-22 12:22:52-05:00   
             frc2473              1 2017-04-22 12:22:52-05:00   

                                 predicted_time       ...         \
key          team_key                                 ...          
2017tur_f1m1 frc2046  2017-04-22 12:19:01-05:00       ...          
             frc1318  2017-04-22 12:19:01-05:00       ...          
             frc1595  2017-04-22 12:19:01-05:00       ...          
             frc1296  2017-04-22 12:19:01-05:00       ...          
             frc2473  2017-04-22 12:19:01-05:00       ...          

                       teleopFuelHigh teleopFuelLow teleopFuelPoints  \
key          team_key                                                  
2017tur_f1m1 frc2046               63             0               21   
             frc1318               63             0               21   
             frc1595               63             0               21   
             frc1296               17             0                5   
             frc2473               17             0                5   

                       teleopPoints  teleopRotorPoints  teleopTakeoffPoints  \
key          team_key                                                         
2017tur_f1m1 frc2046            391                120                  150   
             frc1318            391                120                  150   
             frc1595            391                120                  150   
             frc1296            185                 80                  100   
             frc2473            185                 80                  100   

                       totalPoints      touchpadFar   touchpadMiddle  \
key          team_key                                                  
2017tur_f1m1 frc2046           503  ReadyForTakeoff  ReadyForTakeoff   
             frc1318           503  ReadyForTakeoff  ReadyForTakeoff   
             frc1595           503  ReadyForTakeoff  ReadyForTakeoff   
             frc1296           271  ReadyForTakeoff  ReadyForTakeoff   
             frc2473           271  ReadyForTakeoff  ReadyForTakeoff   

                          touchpadNear  
key          team_key                   
2017tur_f1m1 frc2046   ReadyForTakeoff  
             frc1318   ReadyForTakeoff  
             frc1595   ReadyForTakeoff  
             frc1296              None  
             frc2473              None  

[5 rows x 47 columns]

In [15]:
matches = api.get_matches(sn, "2017tur", "frc1318", response="simple")

In [16]:
matches[50:55]

alliance  surrogate  score  disqualified  \
key          team_key                                            
2017tur_qm43 frc4135      blue      False    319         False   
             frc1318       red      False    277         False   
             frc1523       red      False    277         False   
             frc4206       red      False    277         False   
2017tur_qm52 frc1318      blue      False    253         False   

                                    actual_time comp_level event_key  \
key          team_key                                                  
2017tur_qm43 frc4135  2017-04-20 16:24:27-05:00         qm   2017tur   
             frc1318  2017-04-20 16:24:27-05:00         qm   2017tur   
             frc1523  2017-04-20 16:24:27-05:00         qm   2017tur   
             frc4206  2017-04-20 16:24:27-05:00         qm   2017tur   
2017tur_qm52 frc1318  2017-04-21 08:22:40-05:00         qm   2017tur   

                       match_number            predicted_time  set_number  \
key          team_key                                                       
2017tur_qm43 frc4135             43 2017-04-20 16:24:01-05:00           1   
             frc1318             43 2017-04-20 16:24:01-05:00           1   
             frc1523             43 2017-04-20 16:24:01-05:00           1   
             frc4206             43 2017-04-20 16:24:01-05:00           1   
2017tur_qm52 frc1318             52 2017-04-21 08:22:19-05:00           1   

                                           time winning_alliance  
key          team_key                                             
2017tur_qm43 frc4135  2017-04-20 16:08:00-05:00             blue  
             frc1318  2017-04-20 16:08:00-05:00             blue  
             frc1523  2017-04-20 16:08:00-05:00             blue  
             frc4206  2017-04-20 16:08:00-05:00             blue  
2017tur_qm52 frc1318  2017-04-21 08:24:00-05:00             blue

In [17]:
matches.shape

(102, 12)

In [20]:
matches = api.get_matches(sn, "2017tur", "frc1318", response="keys")

In [21]:
matches.head()

,key
0,2017tur_f1m1
1,2017tur_f1m2
2,2017tur_qf4m1
3,2017tur_qf4m2
4,2017tur_qm100


In [22]:
matches = api.get_matches(sn, team="frc1318", year=2017)

In [23]:
matches.query("score == -1")

alliance  surrogate  score  disqualified actual_time  \
key            team_key                                                        
2017wagg_qf2m2 frc360       blue      False     -1         False         NaT   
               frc3218      blue      False     -1         False         NaT   
               frc5588      blue      False     -1         False         NaT   
               frc2412       red      False     -1         False         NaT   
               frc948        red      False     -1         False         NaT   
               frc1318       red      False     -1         False         NaT   

                        comp_level event_key  match_number post_result_time  \
key            team_key                                                       
2017wagg_qf2m2 frc360           qf  2017wagg             2              NaT   
               frc3218          qf  2017wagg             2              NaT   
               frc5588          qf  2017wagg             2              NaT   
               frc2412          qf  2017wagg             2              NaT   
               frc948           qf  2017wagg             2              NaT   
               frc1318          qf  2017wagg             2              NaT   

                                   predicted_time      ...       \
key            team_key                                ...        
2017wagg_qf2m2 frc360   2017-10-28 19:51:07-05:00      ...        
               frc3218  2017-10-28 19:51:07-05:00      ...        
               frc5588  2017-10-28 19:51:07-05:00      ...        
               frc2412  2017-10-28 19:51:07-05:00      ...        
               frc948   2017-10-28 19:51:07-05:00      ...        
               frc1318  2017-10-28 19:51:07-05:00      ...        

                         teleopFuelHigh teleopFuelLow teleopFuelPoints  \
key            team_key                                                  
2017wagg_qf2m2 frc360               nan           nan              nan   
               frc3218              nan           nan              nan   
               frc5588              nan           nan              nan   
               frc2412              nan           nan              nan   
               frc948               nan           nan              nan   
               frc1318              nan           nan              nan   

                         teleopPoints  teleopRotorPoints  teleopTakeoffPoints  \
key            team_key                                                         
2017wagg_qf2m2 frc360             nan                nan                  nan   
               frc3218            nan                nan                  nan   
               frc5588            nan                nan                  nan   
               frc2412            nan                nan                  nan   
               frc948             nan                nan                  nan   
               frc1318            nan                nan                  nan   

                         totalPoints  touchpadFar  touchpadMiddle  \
key            team_key                                             
2017wagg_qf2m2 frc360            nan          NaN             NaN   
               frc3218           nan          NaN             NaN   
               frc5588           nan          NaN             NaN   
               frc2412           nan          NaN             NaN   
               frc948            nan          NaN             NaN   
               frc1318           nan          NaN             NaN   

                         touchpadNear  
key            team_key                
2017wagg_qf2m2 frc360             NaN  
               frc3218            NaN  
               frc5588            NaN  
               frc2412            NaN  
               frc948             NaN  
               frc1318            NaN  

[6 rows x 47 columns]

In [24]:
matches.tail(6)

alliance  surrogate  score  disqualified  \
key            team_key                                            
2017wasno_qm79 frc1318      blue      False    305         False   
               frc4077      blue      False    305         False   
               frc2976      blue      False    305         False   
               frc3070       red      False    259         False   
               frc4911       red      False    259         False   
               frc949        red      False    259         False   

                                      actual_time comp_level  event_key  \
key            team_key                                                   
2017wasno_qm79 frc1318  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc4077  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc2976  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc3070  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc4911  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc949   2017-03-26 13:24:20-05:00         qm  2017wasno   

                         match_number          post_result_time  \
key            team_key                                           
2017wasno_qm79 frc1318             79 2017-03-26 13:27:03-05:00   
               frc4077             79 2017-03-26 13:27:03-05:00   
               frc2976             79 2017-03-26 13:27:03-05:00   
               frc3070             79 2017-03-26 13:27:03-05:00   
               frc4911             79 2017-03-26 13:27:03-05:00   
               frc949              79 2017-03-26 13:27:03-05:00   

                                   predicted_time       ...         \
key            team_key                                 ...          
2017wasno_qm79 frc1318  2017-03-26 13:25:03-05:00       ...          
               frc4077  2017-03-26 13:25:03-05:00       ...          
               frc2976  2017-03-26 13:25:03-05:00       ...          
               frc3070  2017-03-26 13:25:03-05:00       ...          
               frc4911  2017-03-26 13:25:03-05:00       ...          
               frc949   2017-03-26 13:25:03-05:00       ...          

                         teleopFuelHigh teleopFuelLow teleopFuelPoints  \
key            team_key                                                  
2017wasno_qm79 frc1318         0.000000      0.000000         0.000000   
               frc4077         0.000000      0.000000         0.000000   
               frc2976         0.000000      0.000000         0.000000   
               frc3070        14.000000      0.000000         4.000000   
               frc4911        14.000000      0.000000         4.000000   
               frc949         14.000000      0.000000         4.000000   

                         teleopPoints  teleopRotorPoints  teleopTakeoffPoints  \
key            team_key                                                         
2017wasno_qm79 frc1318     230.000000          80.000000           150.000000   
               frc4077     230.000000          80.000000           150.000000   
               frc2976     230.000000          80.000000           150.000000   
               frc3070     184.000000          80.000000           100.000000   
               frc4911     184.000000          80.000000           100.000000   
               frc949      184.000000          80.000000           100.000000   

                         totalPoints      touchpadFar   touchpadMiddle  \
key            team_key                                                  
2017wasno_qm79 frc1318    305.000000  ReadyForTakeoff  ReadyForTakeoff   
               frc4077    305.000000  ReadyForTakeoff  ReadyForTakeoff   
               frc2976    305.000000  ReadyForTakeoff  ReadyForTakeoff   
               frc3070    259.000000             None  ReadyForTakeoff   
               frc4911    259.000000             None  ReadyForTakeoff   
               frc94

In [25]:
matches.shape

(642, 47)

In [26]:
matches = api.get_matches(sn, team="frc1318", year=2017, response="simple")

In [27]:
matches.head()

alliance  surrogate  score  disqualified  \
key             team_key                                            
2017cmptx_sf1m1 frc2046      blue      False    374         False   
                frc1318      blue      False    374         False   
                frc1595      blue      False    374         False   
                frc987        red      False    493         False   
                frc2122       red      False    493         False   

                                       actual_time comp_level  event_key  \
key             team_key                                                   
2017cmptx_sf1m1 frc2046  2017-04-22 16:42:55-05:00         sf  2017cmptx   
                frc1318  2017-04-22 16:42:55-05:00         sf  2017cmptx   
                frc1595  2017-04-22 16:42:55-05:00         sf  2017cmptx   
                frc987   2017-04-22 16:42:55-05:00         sf  2017cmptx   
                frc2122  2017-04-22 16:42:55-05:00         sf  2017cmptx   

                          match_number            predicted_time  set_number  \
key             team_key                                                       
2017cmptx_sf1m1 frc2046              1 2017-04-22 16:46:01-05:00           1   
                frc1318              1 2017-04-22 16:46:01-05:00           1   
                frc1595              1 2017-04-22 16:46:01-05:00           1   
                frc987               1 2017-04-22 16:46:01-05:00           1   
                frc2122              1 2017-04-22 16:46:01-05:00           1   

                                              time winning_alliance  
key             team_key                                             
2017cmptx_sf1m1 frc2046  2017-04-22 16:00:00-05:00              red  
                frc1318  2017-04-22 16:00:00-05:00              red  
                frc1595  2017-04-22 16:00:00-05:00              red  
                frc987   2017-04-22 16:00:00-05:00              red  
                frc2122  2017-04-22 16:00:00-05:00              red

In [28]:
matches = api.get_matches(sn, team="frc1318", year=2017, response="keys")

In [29]:
matches.head()

,key
0,2017cmptx_sf1m1
1,2017cmptx_sf1m12
2,2017cmptx_sf1m14
3,2017cmptx_sf1m5
4,2017cmptx_sf1m9


In [30]:
matches = api.get_matches(sn, match="2017wasno_qm79")

In [31]:
matches

alliance  surrogate  score  disqualified  \
key            team_key                                            
2017wasno_qm79 frc1318      blue      False    305         False   
               frc4077      blue      False    305         False   
               frc2976      blue      False    305         False   
               frc3070       red      False    259         False   
               frc4911       red      False    259         False   
               frc949        red      False    259         False   

                                      actual_time comp_level  event_key  \
key            team_key                                                   
2017wasno_qm79 frc1318  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc4077  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc2976  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc3070  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc4911  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc949   2017-03-26 13:24:20-05:00         qm  2017wasno   

                         match_number          post_result_time  \
key            team_key                                           
2017wasno_qm79 frc1318             79 2017-03-26 13:27:03-05:00   
               frc4077             79 2017-03-26 13:27:03-05:00   
               frc2976             79 2017-03-26 13:27:03-05:00   
               frc3070             79 2017-03-26 13:27:03-05:00   
               frc4911             79 2017-03-26 13:27:03-05:00   
               frc949              79 2017-03-26 13:27:03-05:00   

                                   predicted_time       ...         \
key            team_key                                 ...          
2017wasno_qm79 frc1318  2017-03-26 13:25:03-05:00       ...          
               frc4077  2017-03-26 13:25:03-05:00       ...          
               frc2976  2017-03-26 13:25:03-05:00       ...          
               frc3070  2017-03-26 13:25:03-05:00       ...          
               frc4911  2017-03-26 13:25:03-05:00       ...          
               frc949   2017-03-26 13:25:03-05:00       ...          

                         teleopFuelHigh teleopFuelLow teleopFuelPoints  \
key            team_key                                                  
2017wasno_qm79 frc1318                0             0                0   
               frc4077                0             0                0   
               frc2976                0             0                0   
               frc3070               14             0                4   
               frc4911               14             0                4   
               frc949                14             0                4   

                         teleopPoints  teleopRotorPoints  teleopTakeoffPoints  \
key            team_key                                                         
2017wasno_qm79 frc1318            230                 80                  150   
               frc4077            230                 80                  150   
               frc2976            230                 80                  150   
               frc3070            184                 80                  100   
               frc4911            184                 80                  100   
               frc949             184                 80                  100   

                         totalPoints      touchpadFar   touchpadMiddle  \
key            team_key                                                  
2017wasno_qm79 frc1318           305  ReadyForTakeoff  ReadyForTakeoff   
               frc4077           305  ReadyForTakeoff  ReadyForTakeoff   
               frc2976           305  ReadyForTakeoff  ReadyForTakeoff   
               frc3070           259             None  ReadyForTakeoff   
               frc4911           259             None  ReadyForTakeoff   
               frc94

In [32]:
matches.shape

(6, 47)

In [33]:
sn.time_zone = "America/Los_Angeles"
matches = api.get_matches(sn, match="2017wasno_qm79", response="simple")

In [34]:
matches

alliance  surrogate  score  disqualified  \
key            team_key                                            
2017wasno_qm79 frc1318      blue      False    305         False   
               frc4077      blue      False    305         False   
               frc2976      blue      False    305         False   
               frc3070       red      False    259         False   
               frc4911       red      False    259         False   
               frc949        red      False    259         False   

                                      actual_time comp_level  event_key  \
key            team_key                                                   
2017wasno_qm79 frc1318  2017-03-26 11:24:20-07:00         qm  2017wasno   
               frc4077  2017-03-26 11:24:20-07:00         qm  2017wasno   
               frc2976  2017-03-26 11:24:20-07:00         qm  2017wasno   
               frc3070  2017-03-26 11:24:20-07:00         qm  2017wasno   
               frc4911  2017-03-26 11:24:20-07:00         qm  2017wasno   
               frc949   2017-03-26 11:24:20-07:00         qm  2017wasno   

                         match_number            predicted_time  set_number  \
key            team_key                                                       
2017wasno_qm79 frc1318             79 2017-03-26 11:25:03-07:00           1   
               frc4077             79 2017-03-26 11:25:03-07:00           1   
               frc2976             79 2017-03-26 11:25:03-07:00           1   
               frc3070             79 2017-03-26 11:25:03-07:00           1   
               frc4911             79 2017-03-26 11:25:03-07:00           1   
               frc949              79 2017-03-26 11:25:03-07:00           1   

                                             time winning_alliance  
key            team_key                                             
2017wasno_qm79 frc1318  2017-03-26 11:26:00-07:00             blue  
               frc4077  2017-03-26 11:26:00-07:00             blue  
               frc2976  2017-03-26 11:26:00-07:00             blue  
               frc3070  2017-03-26 11:26:00-07:00             blue  
               frc4911  2017-03-26 11:26:00-07:00             blue  
               frc949   2017-03-26 11:26:00-07:00             blue